# Data downloading and preprocessing

Let's download an example dataset from MOABB and preprocess it

In [4]:
# Specify the path to the folder containing the data
path = None # ex: '/Brain/private/y17eloua/tmp_reve'

In [ ]:
from moabb.datasets import PhysionetMI
from moabb.paradigms import LeftRightImagery
import numpy as np
import pandas as pd
import mne
from sklearn.preprocessing import LabelEncoder
from os.path import join as pjoin
import os
from einops import rearrange
import gc 
import torch
from scipy.linalg import sqrtm, inv,pinv

def compute_r_op(X):
    r = torch.einsum('bet, tab -> bea',X,X.T).mean(0)
    r_op = torch.from_numpy(inv(sqrtm(r)))
    return r_op

def load_data(dset,num_classes,selected_chans = None,subjects=None,sfreq = 160,fmin=0.5,fmax=63.99,tmin=0.5,tmax=3.5):
    ds_src = dset()
    if dset.__name__ == 'PhysionetMI':
        ds_src.event_id = dict(left_hand=2, right_hand=3, feet=5, hands=4)
    src_prgm = LeftRightImagery(channels=selected_chans, resample=sfreq, fmin=fmin, fmax=fmax, tmin=tmin,tmax=tmax)
    if not subjects:
        epoch_X_src, label_src, m_src = src_prgm.get_data(dataset=ds_src, return_epochs=True)
    else : 
        epoch_X_src, label_src, m_src = src_prgm.get_data(dataset=ds_src,subjects=subjects, return_epochs=True)
        
    encoder_subject = LabelEncoder()
    subject_rename_dict = dict(zip(m_src.subject.unique(),encoder_subject.fit_transform(m_src.subject.unique())))
    encoder_session = LabelEncoder()                        
    session_rename_dict = dict(zip(m_src.session.unique(),encoder_session.fit_transform(m_src.session.unique())))
    m_src['subject'] = m_src['subject'].map(subject_rename_dict)
    m_src['session'] = m_src['session'].map(session_rename_dict) 
    m_src = m_src.reset_index().rename(columns={'index':'trial'})
    m_src.to_csv(os.path.join(path,dset_name_path+'/'+dset_name_path+'_m.csv'),index=False)
    return epoch_X_src,label_src,m_src

def prepro_dataset(dataset,dset_name_path,n_c,t_max):
    subjects = [int(x) for x in list(np.arange(1,110)) if x not in [38,88,89,92,100,104]] 
    f_max = 63.99
    epoch_X,label_src,m_src = load_data(dataset,n_c,
                                          sfreq = 128,
                                          fmin=0.,
                                          fmax=f_max,
                                          tmin=0.5,tmax=3.5,
                                          subjects=subjects)
    epoch_X.resample(200)
    encoder = LabelEncoder()
    Y = encoder.fit_transform(label_src)
    X = epoch_X.get_data().astype('float32')
    
    modulo = -(X.shape[-1]%50)
    if modulo !=0:
        X = X[:,:,:modulo]

    valid_chans = mne.channels.make_standard_montage("standard_1005").get_positions()['ch_pos']
    chan_to_keep = sorted([ch for ch in epoch_X.ch_names if ch in list(valid_chans.keys())])       
    epoch_X.pick(chan_to_keep)
    channels = epoch_X.ch_names
    positions = [ valid_chans[ch] for ch in  channels]
    positions = np.array(positions).astype('float32')
    channels = np.array(channels)


    X_n,X_c,X_t = X.shape
    print(X.shape)
    save_path = os.path.join(path,dset_name_path)
    big_recording_filename = pjoin(save_path,'X_-_eeg_-_'+dset_name_path+'.npy')
    big_save = np.memmap(big_recording_filename,dtype='float32',mode='w+',shape=(X_n,X_c,X_t))
    big_save[:] = X

    Y_recording_filename = pjoin(save_path,'Y_-_eeg_-_'+dset_name_path+'.npy')
    Y_save = np.memmap(Y_recording_filename,dtype='int64',mode='w+',shape=(X_n))
    Y_save[:] = Y

    np.save(pjoin(save_path,'pos_-_eeg_-_'+dset_name_path+'.npy'),positions.astype('float32'))

    big_save.flush()
    Y_save.flush()
    del big_save,Y_save
    gc.collect()

    z_score_list,ea_list = [],[]
    for s_id in sorted(m_src.subject.unique()):
        m_src_i = m_src[m_src['subject']==s_id]
        z_score_list_tmp, ea_list_tmp =  [],[]
        for session in sorted(m_src_i.session.unique()):
            idx = m_src_i[m_src_i['session']==session].trial.tolist()
            X_s = X[idx]
            mean = rearrange(X_s, 'b c t -> (b t) c').mean(axis = 0)
            std = rearrange(X_s, 'b c t -> (b t) c').std(axis = 0)        
            sqrt_R_s = compute_r_op(torch.from_numpy(X_s)).numpy()
            torch.einsum("fe,bet->bft",torch.from_numpy(sqrt_R_s), torch.from_numpy(X_s))
            X_EA_s = torch.einsum("fe,bet->bft",torch.from_numpy(sqrt_R_s), torch.from_numpy(X_s)).numpy()
            EA_mean = rearrange(X_EA_s, 'b c t -> (b t) c').mean(axis = 0)
            EA_std  = rearrange(X_EA_s, 'b c t -> (b t) c').std(axis = 0)
            z_score_list_tmp.append(np.array([mean,std]))
            ea_list_tmp.append(np.concatenate([sqrt_R_s,np.array([EA_mean,EA_std])]))
        z_score_list.append(z_score_list_tmp)
        ea_list.append(ea_list_tmp)

    np.save(pjoin(save_path,'stats_-_zscore_-_'+dset_name_path+'.npy'),np.array(z_score_list))
    np.save(pjoin(save_path,'stats_-_EA_-_'+dset_name_path+'.npy'),np.array(ea_list))


    
datasets  = [PhysionetMI]
classes = [2]
durations = [3]

for d,n_c,t_max in zip(datasets,classes,durations):
    dset_name_path = str(d).split('.')[-1].split("'>")[0]
    print(dset_name_path,n_c,t_max)
    if not os.path.exists(os.path.join(path,dset_name_path)):
        os.mkdir(os.path.join(path,dset_name_path))
    prepro_dataset(d,dset_name_path,n_c,t_max)
        

PhysionetMI 2 3
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['feet', 'hands', 'rest']


/Brain/private/y17eloua/env_fsreve/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 15 events (all good), 0.5 – 3.5 s (baseline off), ~3.6 MiB, data loaded,
 'left_hand': 8
 'right_hand': 7>
  warn(f"warnEpochs {epochs}")
/Brain/private/y17eloua/env_fsreve/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 15 events (all good), 0.5 – 3.5 s (baseline off), ~3.6 MiB, data loaded,
 'left_hand': 8
 'right_hand': 7>
  warn(f"warnEpochs {epochs}")
/Brain/private/y17eloua/env_fsreve/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 15 events (all good), 0.5 – 3.5 s (baseline off), ~3.6 MiB, data loaded,
 'left_hand': 7
 'right_hand': 8>
  warn(f"warnEpochs {epochs}")


Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['feet', 'hands', 'rest']


/Brain/private/y17eloua/env_fsreve/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 15 events (all good), 0.5 – 3.5 s (baseline off), ~3.6 MiB, data loaded,
 'left_hand': 7
 'right_hand': 8>
  warn(f"warnEpochs {epochs}")
/Brain/private/y17eloua/env_fsreve/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 15 events (all good), 0.5 – 3.5 s (baseline off), ~3.6 MiB, data loaded,
 'left_hand': 8
 'right_hand': 7>
  warn(f"warnEpochs {epochs}")
/Brain/private/y17eloua/env_fsreve/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 15 events (all good), 0.5 – 3.5 s (baseline off), ~3.6 MiB, data loaded,
 'left_hand': 8
 'right_hand': 7>
  warn(f"warnEpochs {epochs}")


Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['feet', 'hands', 'rest']


...

/Brain/private/y17eloua/env_fsreve/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 15 events (all good), 0.5 – 3.5 s (baseline off), ~3.6 MiB, data loaded,
 'left_hand': 7
 'right_hand': 8>
  warn(f"warnEpochs {epochs}")
/Brain/private/y17eloua/env_fsreve/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 15 events (all good), 0.5 – 3.5 s (baseline off), ~3.6 MiB, data loaded,
 'left_hand': 8
 'right_hand': 7>
  warn(f"warnEpochs {epochs}")
/Brain/private/y17eloua/env_fsreve/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 15 events (all good), 0.5 – 3.5 s (baseline off), ~3.6 MiB, data loaded,
 'left_hand': 8
 'right_hand': 7>
  warn(f"warnEpochs {epochs}")


Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding m

/Brain/private/y17eloua/env_fsreve/lib/python3.12/site-packages/moabb/paradigms/base.py:350: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  X = mne.concatenate_epochs(X)


Adding metadata with 3 columns
4635 matching events found
No baseline correction applied
(4635, 64, 600)


/tmp/ipykernel_2066956/4096202077.py:15: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3725.)
  r = torch.einsum('bet, tab -> bea',X,X.T).mean(0)
